<a href="https://colab.research.google.com/github/KrunalVaghela62/newww/blob/main/nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from std_msgs.msg import Float32MultiArray  # For publishing an array of centers
from cv_bridge import CvBridge
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Use cv2_imshow for Colab

class ImageSubscriber(Node):
    def __init__(self):
        super().__init__('image_subscriber')

        # Subscribe to the /image_raw topic where images are being published
        self.subscription = self.create_subscription(
            Image,  # Message type
            '/image_raw',  # Topic name (to match the publisher topic)
            self.listener_callback,  # Callback function to process the image
            10  # QoS (Quality of Service) policy
        )

        # Publisher to publish center points to the /u_v topic
        self.publisher_ = self.create_publisher(
            Float32MultiArray,  # The message type to publish the centers
            '/u_v',  # The topic where center coordinates will be published
            10
        )

        self.bridge = CvBridge()

    def listener_callback(self, msg):
        # Convert the ROS Image message to an OpenCV image
        frame = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # Define the color in BGR colorspace (yellow in this case)
        yellow = [255, 0, 157]

        # Precompute the HSV range for the target color
        def get_precomputed_limits(color):
            c = np.uint8([[color]])  # Convert to HSV
            hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
            hue = hsvC[0][0][0]

            # Handle wrap-around for hues
            lower_limit = np.array([max(0, hue - 10), 100, 100], dtype=np.uint8)
            upper_limit = np.array([min(180, hue + 10), 255, 255], dtype=np.uint8)

            return lower_limit, upper_limit

        # Get HSV bounds for yellow
        lower_limit, upper_limit = get_precomputed_limits(yellow)

        # Convert the image to HSV colorspace
        hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create a mask for the specified color (yellow)
        mask = cv2.inRange(hsv_image, lower_limit, upper_limit)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Loop over the contours and draw bounding boxes around them
        centers = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)

            # Optionally, filter small contours by area
            if w * h > 10:  # Set a minimum area threshold
                # Draw the bounding box on the original frame
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 5)
                centers.append((x + (w // 2), y + (h // 2)))
        centers = np.array(centers)

        # Display the processed image and print the center coordinates
        cv2_imshow(frame)
        center_msg = Float32MultiArray()
        # Flatten the centers array and assign it to the message data
        center_msg.data = centers.flatten().tolist()
        self.publisher_.publish(center_msg)
        self.get_logger().info(f"Published centers: {centers.flatten()}")

def main(args=None):
    rclpy.init(args=args)
    image_subscriber = ImageSubscriber()

    rclpy.spin(image_subscriber)

    # Clean up
    image_subscriber.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()


In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import Float32MultiArray  # Message type for the center points
from geometry_msgs.msg import Point  # Message type for 3D world coordinates (X, Y, Z)
import numpy as np

def pixel_to_world(u, v, depth, K):
    # Inverse of the intrinsic matrix
    inv_K = np.linalg.inv(K)

    # Convert pixel to normalized camera coordinates
    pixel_coords = np.array([u, v, 1])  # Homogeneous image coordinates
    normalized_coords = inv_K @ pixel_coords  # Apply inverse intrinsic matrix

    # Scale normalized coordinates by depth to get camera coordinates
    camera_coords = normalized_coords * depth  # [X_c, Y_c, Z_c]

    # Extract world coordinates
    X_w, Y_w, Z_w = camera_coords[:3]
    return X_w, Y_w, Z_w


class CenterSubscriber(Node):
    def __init__(self):
        super().__init__('center_subscriber')

        # Subscribe to the /u_v topic where the centers are being published
        self.subscription = self.create_subscription(
            Float32MultiArray,  # Message type for the center points
            '/u_v',  # Topic name (to match the publisher topic)
            self.listener_callback,  # Callback function to process the data
            10  # QoS (Quality of Service) policy
        )

        # Publisher to send the computed world coordinates to /motion_p topic
        self.motion_publisher = self.create_publisher(
            Point,  # The message type for the 3D world coordinates
            '/motion_p',  # The topic where world coordinates will be published
            10
        )

        # Example intrinsic camera matrix K
        self.K = np.array([
            [667.248390, 0.000000 ,644.165204],
            [0.000000 ,668.783677, 417.422792],
            [0.000000 ,0.000000 ,1.000000]
        ])

    def listener_callback(self, msg):
        # Extract center points (flattened array)
        centers = np.array(msg.data).reshape(-1, 2)

        # Assuming random depths for example (replace with real depth data if available)
        depths = np.random.rand(len(centers), 1) * 10  # Random depths between 0 and 10 meters

        # Process each center and compute world coordinates
        for (u, v), depth in zip(centers, depths):
            X_w, Y_w, Z_w = pixel_to_world(u, v, depth, self.K)

            # Log the world coordinates
            self.get_logger().info(f"World Coordinates: X={X_w}, Y={Y_w}, Z={Z_w}")

            # Create a Point message to publish the world coordinates
            world_point = Point()
            world_point.x = X_w
            world_point.y = Y_w
            world_point.z = Z_w

            # Publish the world coordinates to the /motion_p topic
            self.motion_publisher.publish(world_point)
            self.get_logger().info(f"Published world point: {world_point}")

def main(args=None):
    rclpy.init(args=args)
    center_subscriber = CenterSubscriber()

    rclpy.spin(center_subscriber)

    # Clean up
    center_subscriber.destroy_node()
    rclpy.shutdown()


if __name__ == '__main__':
    main()
